In [1]:
import datetime as dt
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from random import betavariate, random, randrange, seed, uniform
import requests
import seaborn as sns
import sqlite3

seed(9)

## SQL Queries

The following code makes a SQL database in SQLite from two sets of data, customers and orders. Since the `CustomerId` field is randomly generated with a [Beta distribution](https://en.wikipedia.org/wiki/Beta_distribution), many customers have no orders and some have many orders.

In [2]:
customers = [(i, f'Customer {i}', f'customer{i}@example.com') for i in range(1, 101)]
types = ['commercial', 'industrial', 'residential']
orders = [(i, int(100*betavariate(2, 5)+1), (dt.datetime.now() - dt.timedelta(60 * random())).strftime('%Y-%m-%d'), types[randrange(len(types))], round(uniform(10, 500), 2))
          for i in range(1, 301)]

print(f'Generated {len(customers)} customers and {len(orders)} orders.')

# Display several orders spread across the data to check the structure.
len(orders), orders[::50]

Generated 100 customers and 300 orders.


(300,
 [(1, 41, '2025-12-15', 'residential', 237.22),
  (51, 29, '2025-10-18', 'commercial', 23.77),
  (101, 46, '2025-11-23', 'commercial', 174.13),
  (151, 21, '2025-11-13', 'residential', 188.09),
  (201, 16, '2025-12-06', 'commercial', 146.33),
  (251, 16, '2025-10-26', 'industrial', 457.45)])

In [3]:
# Make an in-memory SQLite database.
conn = sqlite3.connect(':memory:')

# Make a Pandas DataFrame for the customers and add its data to a SQL table.
customers_df = pd.DataFrame(customers)
customers_df.columns = ['CustomerId', 'CustomerName', 'EmailAddress']
customers_df.to_sql('customers', conn, if_exists='replace', index=False)

# Make a Pandas DataFrame for the orders and add its data to a SQL table.
orders_df = pd.DataFrame(orders)
orders_df.columns = ['OrderId', 'CustomerId', 'OrderDate', 'OrderType', 'OrderAmount']
orders_df.to_sql('orders', conn, if_exists='replace', index=False)

# Inspect the tables.
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type = 'table';")
tables = cursor.fetchall()
print('Tables in database:', tables)

# Show the first few rows of the 'customers' table.
print("First three rows of 'customers' table:")
query_customers = pd.read_sql_query("SELECT * FROM customers LIMIT 3;", conn)
display(query_customers)

# Show the first few rows of the 'orders' table.
print("First three rows of 'orders' table:")
query_orders = pd.read_sql_query("SELECT * FROM orders LIMIT 3;", conn)
display(query_orders)

Tables in database: [('customers',), ('orders',)]
First three rows of 'customers' table:


,CustomerId,CustomerName,EmailAddress
0,1,Customer 1,customer1@example.com
1,2,Customer 2,customer2@example.com
2,3,Customer 3,customer3@example.com


First three rows of 'orders' table:


,OrderId,CustomerId,OrderDate,OrderType,OrderAmount
0,1,41,2025-12-15,residential,237.22
1,2,27,2025-10-19,residential,217.07
2,3,10,2025-10-23,industrial,110.43


### Joining and aggregating

The following code joins the two tables and aggregates the order amount by city.

In [4]:
sql_query = """
SELECT c.CustomerId, c.CustomerName, c.EmailAddress, o.OrderDate, o.OrderType, o.OrderAmount
FROM customers AS c
INNER JOIN orders AS o
ON c.CustomerId = o.CustomerId;
"""
joined_df = pd.read_sql_query(sql_query, conn)
print(f'{len(joined_df)} rows')
joined_df[::50]

300 rows


,CustomerId,CustomerName,EmailAddress,OrderDate,OrderType,OrderAmount
0,1,Customer 1,customer1@example.com,2025-10-23,industrial,404.29
50,13,Customer 13,customer13@example.com,2025-10-19,commercial,92.67
100,20,Customer 20,customer20@example.com,2025-10-22,industrial,178.44
150,28,Customer 28,customer28@example.com,2025-11-12,commercial,75.22
200,36,Customer 36,customer36@example.com,2025-10-23,residential,420.37
250,46,Customer 46,customer46@example.com,2025-11-30,residential,42.48


In [5]:
sql_query = """
SELECT c.CustomerId, c.CustomerName, c.EmailAddress, COUNT(o.OrderAmount) as OrderCount, SUM(o.OrderAmount) as OrderSum
FROM customers AS c
INNER JOIN orders AS o
ON c.CustomerId = o.CustomerId
GROUP BY c.CustomerId, c.CustomerName, c.EmailAddress;
"""
aggregated_df = pd.read_sql_query(sql_query, conn)
print(f'{len(aggregated_df)} rows')
aggregated_df

67 rows


,CustomerId,CustomerName,EmailAddress,OrderCount,OrderSum
0,1,Customer 1,customer1@example.com,1,404.29
1,3,Customer 3,customer3@example.com,2,787.72
2,4,Customer 4,customer4@example.com,3,300.95
3,5,Customer 5,customer5@example.com,4,566.30
4,6,Customer 6,customer6@example.com,4,420.28
...,...,...,...,...,...
62,70,Customer 70,customer70@example.com,2,393.85
63,76,Customer 76,customer76@example.com,1,383.19
64,77,Customer 77,customer77@example.com,1,353.31
65,78,Customer 78,customer78@example.com,1,126.00


## API Ingestion

The following code uses a public API to get the UCI `adult` dataset and persists the data as a CSV file.

In [11]:
import os
from urllib.request import urlretrieve
url = 'https://huggingface.co/datasets/scikit-learn/adult-census-income/resolve/main/adult.csv'
file_name = os.path.basename(url)
if not os.path.exists(file_name):
    urlretrieve(url, file_name)
with open(file_name, 'rt') as fin:
    print(*fin.readlines()[:9], sep='')

"age","workclass","fnlwgt","education","education.num","marital.status","occupation","relationship","race","sex","capital.gain","capital.loss","hours.per.week","native.country","income"
90,"?",77053,"HS-grad",9,"Widowed","?","Not-in-family","White","Female",0,4356,40,"United-States","<=50K"
82,"Private",132870,"HS-grad",9,"Widowed","Exec-managerial","Not-in-family","White","Female",0,4356,18,"United-States","<=50K"
66,"?",186061,"Some-college",10,"Widowed","?","Unmarried","Black","Female",0,4356,40,"United-States","<=50K"
54,"Private",140359,"7th-8th",4,"Divorced","Machine-op-inspct","Unmarried","White","Female",0,3900,40,"United-States","<=50K"
41,"Private",264663,"Some-college",10,"Separated","Prof-specialty","Own-child","White","Female",0,3900,40,"United-States","<=50K"
34,"Private",216864,"HS-grad",9,"Divorced","Other-service","Unmarried","White","Female",0,3770,45,"United-States","<=50K"
38,"Private",150601,"10th",6,"Separated","Adm-clerical","Unmarried","White","Male",0,3770,40,"

## Data Cleaning

The UCI `adult` dataset has both categorical and numerical columns. Many of the values are missing, replaced with a `?`.